In [21]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [22]:
# 데이터 불러오기 
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [23]:
print(train_df.shape)
# (batch,컬럼) 형태의 구조
train_df

(60000, 786)


,index,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,6,0,0,0,0,0,0,0,5,...,0,0,0,30,43,0,0,0,0,0
3,3,0,0,0,0,1,2,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,59996,1,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,59997,8,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,59998,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
test_df

,index,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,9996,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,9997,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,9998,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [25]:
#학습에 사용하기 위해서는 float형태로 바뀌어야 한다.
train_data = np.array(train_df.iloc[:,1:], dtype = 'float32') #인덱스를 제외하고 넣기
test_data = np.array(test_df.iloc[:,1:], dtype='float32')

In [26]:
x_train = train_data[:, 1:] # 피처들
y_train = train_data[:,0]  # 라벨

x_test = test_data

In [27]:
#validation set 두기 (정확한 평가를 위해서)
#주로 하이퍼 파라미터 찾기 위해서 validation set을 둔다.
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)

print(x_train.shape, x_validate.shape)


(48000, 784) (12000, 784)


In [45]:
#모델 구축
from keras.models import Sequential
from keras.layers import Dense
nn_model = Sequential([
    Dense(728,),
    Dense(364),# 728->364
    Dense(364),
    Dense(182),
    Dense(10,activation='softmax')
])
#feature는 아니지만, 깊게 노드를 연결하면 스스로 feature를 추출할 수 있게 된다는 뜻.

In [48]:
#compile이란 우리가 만든 모델을 어떻게 weight들을 최적화 시킬지 정해주는 부분.
from keras.optimizers import Adam
nn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])
#Adam은 가중치를 학습 시켜주는 optimizer중 가장 유명한 것.
#learning rate는 데이터, 전처리 방법에 따라 달라진다. 스스로 조절해가며 찾아야한다.
#metrics는 학습하면서 평가해주는 중간결과를 보여줄 때 사용.

#sparse_categorical_crossentropy는 0~10으로된 라벨를 10개로 만들어진 백터와 매칭시켜준다.

#categorical_crossentropy를 쓰면,0~10칸으로 된 라벨을 10개 벡터와 매칭.


In [51]:
nn_model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=30,
    verbose=1,
    validation_data=(x_validate,y_validate),
)

Epoch 1/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.5210 - accuracy: 0.8314 - val_loss: 0.6338 - val_accuracy: 0.8222
Epoch 2/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.5612 - accuracy: 0.8174 - val_loss: 0.7544 - val_accuracy: 0.7536
Epoch 3/30
1500/1500 [==============================] - 7s 5ms/step - loss: 45.0619 - accuracy: 0.7664 - val_loss: 1.3578 - val_accuracy: 0.8001
Epoch 4/30
1500/1500 [==============================] - 9s 6ms/step - loss: 0.8712 - accuracy: 0.8174 - val_loss: 0.7307 - val_accuracy: 0.8252
Epoch 5/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5849 - accuracy: 0.8289 - val_loss: 0.5216 - val_accuracy: 0.8356
Epoch 6/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.5374 - accuracy: 0.8303 - val_loss: 0.5387 - val_accuracy: 0.8211
Epoch 7/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.5733 - accuracy: 0.8179 - val_loss: 0.7478 - val_accuracy

In [62]:
y_pred = nn_model.predict(x_test)

313/313 [==============================] - 1s 2ms/step


In [67]:
y_pred.shape
#10칸에 대해, 확률들을 적어 둔 것.

(10000, 10)

array([0, 1, 6, ..., 8, 6, 1], dtype=int64)

In [71]:

submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8')
submission['label'] = y_pred.argmax(axis=1)
submission.to_csv('fashion_submission.csv', index = False)

In [72]:
submission['label'].max()

9

In [73]:
#이미지로 확인하기.
submission

,index,label
0,0,0
1,1,1
2,2,6
3,3,0
4,4,3
...,...,...
9995,9995,0
9996,9996,6
9997,9997,8
9998,9998,6
